In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from shapely import Point, Polygon
import geopandas as gpd
import folium
import sys

In [6]:
# Get the absolute path of the utils.py directory
utils_path = os.path.abspath("../demand_estimation/")
sys.path.append(utils_path)

# Now import the module
from utils_data import get_contours_city_ign

In [7]:
code_insee = '44055'

# Pour avoir les bornes au sein de la ville
file_ign = '../Data/BDCARTO/44_Loire_Atlantique/data.gpkg'
file_bornes = '../data/consolidation-etalab-schema-irve-statique-v-2.3.1-20250129.csv'

# Pour faire les calculs et visualiser
crs_meters = 'EPSG:2154'
crs_angles = 'EPSG:4326'

# Identifiant qui permet de savoir quelles bornes appartiennent à Vinci / Easycharge
vinci_names_operateur = ['Easy Charge | FR*ECH', 'Easycharge services']

# Loading et visualisation des bornes dans la ville

In [8]:
df_bornes = pd.read_csv(file_bornes)

# Convert latitude & longitude to a geometry column
df_bornes['geometry'] = df_bornes.apply(lambda row: Point(row['consolidated_longitude'], row['consolidated_latitude']), axis=1)
gdf_bornes = gpd.GeoDataFrame(df_bornes, geometry='geometry')
gdf_bornes.set_crs(crs_angles, inplace=True)
gdf_bornes.head(3)

C:\Users\eliot\AppData\Local\Temp\ipykernel_7076\1686685931.py:1: DtypeWarning: Columns (12,18,19,20,21,22,24,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bornes = pd.read_csv(file_bornes)


,nom_amenageur,siren_amenageur,contact_amenageur,nom_operateur,contact_operateur,telephone_operateur,nom_enseigne,id_station_itinerance,id_station_local,nom_station,...,datagouv_resource_id,datagouv_organization_or_owner,created_at,consolidated_longitude,consolidated_latitude,consolidated_code_postal,consolidated_commune,consolidated_is_lon_lat_correct,consolidated_is_code_insee_verified,geometry
0,SOLVEO ENERGIES,508886132.0,bornes@solveo-energies.com,SOLVEO ENERGIES,bornes@solveo-energies.com,05 32 98 01 58,SOLVEO ENERGIES,DKMONE3785539,NaN,Golf de Garonne,...,a9352425-5392-4028-b02d-fcc88b856a0e,solveo-energies,2024-12-19T10:53:36.428000+00:00,1.407644,43.628261,NaN,Toulouse,True,True,POINT (1.40764 43.62826)
1,SOLVEO ENERGIES,508886132.0,bornes@solveo-energies.com,SOLVEO ENERGIES,bornes@solveo-energies.com,05 32 98 01 58,SOLVEO ENERGIES,DKMONE3785539,NaN,Golf de Garonne,...,a9352425-5392-4028-b02d-fcc88b856a0e,solveo-energies,2024-12-19T10:53:36.428000+00:00,1.407644,43.628261,NaN,Toulouse,True,True,POINT (1.40764 43.62826)
2,SOLVEO ENERGIES,508886132.0,bornes@solveo-energies.com,SOLVEO ENERGIES,bornes@solveo-energies.com,05 32 98 01 58,SOLVEO ENERGIES,DKMONE3785539,NaN,Golf de Garonne,...,a9352425-5392-4028-b02d-fcc88b856a0e,solveo-energies,2024-12-19T10:53:36.428000+00:00,1.407644,43.628261,NaN,Toulouse,True,True,POINT (1.40764 43.62826)


In [9]:
# Getting data for the considered city
gdf_contours_city, geometry_contours_city = get_contours_city_ign(file_ign, code_insee, crs=crs_angles)
center_lon, center_lat = geometry_contours_city.centroid.coords[0]
gdf_bornes_city = gdf_bornes[gdf_bornes.geometry.intersects(geometry_contours_city)]

# Ajout d'un booléen qui indique si la borne appartient à Vinci ou non
gdf_bornes_city['bornes_vinci'] = gdf_bornes_city['nom_operateur'].isin(vinci_names_operateur).astype(int)
gdf_bornes_city.head()

c:\Users\eliot\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,nom_amenageur,siren_amenageur,contact_amenageur,nom_operateur,contact_operateur,telephone_operateur,nom_enseigne,id_station_itinerance,id_station_local,nom_station,...,datagouv_organization_or_owner,created_at,consolidated_longitude,consolidated_latitude,consolidated_code_postal,consolidated_commune,consolidated_is_lon_lat_correct,consolidated_is_code_insee_verified,geometry,bornes_vinci
12331,ChargeGuru,NaN,info@chargeguru.com,ChargeGuru,info@chargeguru.com,+33176341178,EPAF VACANCES LA BAULE - 44500 - 1,FRCG0E001287,FRCG0E001287,EPAF VACANCES LA BAULE - 44500 - 1,...,eco-movement,2023-06-28T11:46:08.539000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
12332,RAIDEN SAS (ChargeGuru),NaN,NaN,ChargePoint Austria GmbH | FR*CG0,cpo-tech@chargeguru.com,NaN,ChargeGuru,FRCG0P1208388749112575279,978730,ChargeGuru/3ad12c64-283f-4154-adc0-1ebda5c24410,...,gireve-2,2023-03-24T14:32:54.036000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
12419,ChargeGuru,NaN,info@chargeguru.com,ChargeGuru,info@chargeguru.com,+33176341178,EPAF VACANCES LA BAULE - 44500 - 1,FRCG0E001466,FRCG0E001466,EPAF VACANCES LA BAULE - 44500 - 1,...,eco-movement,2023-06-28T11:46:08.539000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
12420,RAIDEN SAS (ChargeGuru),NaN,NaN,ChargePoint Austria GmbH | FR*CG0,cpo-tech@chargeguru.com,NaN,ChargeGuru,FRCG0P1208388749112575279,978730,ChargeGuru/3ad12c64-283f-4154-adc0-1ebda5c24410,...,gireve-2,2023-03-24T14:32:54.036000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
22926,EasyCharge Services,901772400.0,exploitation@easycharge-vinci.com,Easycharge services,exploitation@easycharge-vinci.COM,3 67 47 06 40,Easycharge services,FRECHPN1QJPHUMBX,N1QJPHUMBX,"La Baule , Pkg Atlantia - Palais des Festivals",...,easycharge-services,2022-12-27T17:28:14.243000+00:00,-2.409128,47.284032,NaN,NaN,False,False,POINT (-2.40913 47.28403),1


In [10]:
# Create a folium map centered on the centroid
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    geometry_contours_city,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

for idx, row in gdf_bornes_city.iterrows():
    popup_text = f"""
    <strong>Charging Station</strong><br>
    Nb pdc: {row['nbre_pdc']}<br>
    Amenageur: {row['nom_amenageur']}<br>
    Operateur: {row['nom_operateur']}
    """
    marker_color = 'red' if row['nom_operateur'] in vinci_names_operateur else 'blue'
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=popup_text,
        icon=folium.Icon(color=marker_color)
    ).add_to(m)
# Save the map to an HTML file or display it
# m.save("map_france_data.html")
m